## Load Conversations

In [58]:
import sys
sys.path.append('.')

import pandas as pd
from utils import load_jsonl, dump_jsonl
# import numpy as np

In [59]:
def get_task1_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    
    def to_message_str(messages, users):
        s = []
        u = []
        for m in messages:
            if users[m['user_id']] == "SYS":
                s.append(m['text'])
            else:
                u.append(m['text'])
                
        return s, u
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        chunk_size = 100
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            s, u = to_message_str(sub_messages, users)
            
            if pd.isna(row[col_label]):
                continue
            
            if row[col_label] in skips:
                continue
                
            label = row[col_label]
                
            newdata.append({
                "user": u,
                "sys": s,
                "label": label,
                "nturn": len(sub_messages)
            })
        
#     n_val = int(len(newdata)*0.05)
#     n_test = n_val
    
#     test = newdata[0:n_test]
#     val = newdata[n_test:n_test+n_val]
#     train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [60]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
df.head()

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",1. Close,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",1. Close,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [61]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,551,551,551
2. Know each other,230,230,230
3. Don't know each other,435,435,435
4. Don't like each other,5,5,5


In [62]:
# df.groupby("label").count()

In [63]:
def get_task2_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["created_at"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())!=0 else "SYS"
                users[m["user_id"]] = username
                
        if len(users)>2:
            print("More than 1 users", len(users))
        
        
        
        messages = row["messages"]
        s = []
        u = []
        for m in messages:
            text = m['text'].replace("[USR]", "").replace("[URL]", "URL")
            if users[m['user_id']] == "SYS":
                s.append(text)
            else:
                u.append(text)
        
        label = row[col_label]
            
        if pd.isna(label):
            continue
        
        if label in skips:
            continue
                
        newdata.append({
            "user": u,
            "sys": s,
            "label": label,
            "nturn": len(messages)
        })
                
            
    n_val = int(len(newdata)*0.1)
    n_test = n_val
    
    test = newdata[0:n_test]
    val = newdata[n_test:n_test+n_val]
    train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [64]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority")
df.head()

Loaded 2486 records from ./Task2/annotated/annotated.jsonl


,user,sys,label,nturn
0,[ แตงโม ชวนแฟนไปค่ะ ไม่ใช่กระติกชวนไป กลับไปหา...,[ทบทวน\nงานคืนนั้น #กระติก ชวนแฟนพี่ตม.ไปด้วยน...,3. Not respect,3
1,[ รูปนี้จ้า ยัยน้องตัวเล็กของพี่มิว☀️🌻\n \n#...,[เนื่ิองจากวันนี้...เป็นวันครบรอบ 3 ปี\nมาเล่น...,1. Respect,3
2,[ แอบคิดเหมือนกัน้ลยค่ะว่าจะมีอีกรอบ],[เธรดนี้จะมาวิเคราะห์น้ำท่วมปี54 ที่จริงมันอาจ...,2. Normal,3
3,[ เนี่ยสิ่งที่กูพูด URL],[มาดิ้นควยไรชองมึง\nมึงอ่ะโชว์โง่ไอ้สัตว์ เสร่...,3. Not respect,3
4,"[ สรุปปวินนี่บล็อกคนด้วยมั้ย?, สรุป ที่น่าโมโ...",[แต่ปวินกับไพรวัลย์ก็ไม่เข้าใจนะเหมือนเจ้เกลีย...,2. Normal,5


In [65]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Respect,319,319,319
2. Normal,1661,1661,1661
3. Not respect,364,364,364


In [66]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness")
df.head()

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",2. Know each other,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",2. Know each other,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [67]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,462,462,462
2. Know each other,696,696,696
3. Don't know each other,52,52,52
4. Don't like each other,11,11,11


## Load Lexicons

In [68]:
from pythainlp.tokenize import word_tokenize

In [69]:
import json
with open("./PrivateSpace/thai-dictionary/RoyalInstituteDictionary/words.json", encoding="utf-8") as fin:
    raw = json.load(fin)
    thaidict_royal = set()
    for k in raw:
        thaidict_royal.update(raw[k])

In [70]:
lexicons_arr = load_jsonl("lexicons.jsonl")

Loaded 25573 records from lexicons.jsonl


In [71]:
from collections import defaultdict
tags = set()
lexicons = {}
lexicons_keys = defaultdict(list)

for key, values  in lexicons_arr:
    if len(key) <= 1:
        continue
        
    key = key.lower()
    if key.endswith("rep"):
        key = key.replace("rep", "")
        
    w = word_tokenize(key)
    
    lexicons_keys[w[0]].append(key)
    
    tag = [t for t in values["tags"] if not t.startswith("cat:")]
    lexicons[key] = tag
    tags.update(tag)

In [72]:
from pythainlp.util import countthai

In [74]:
import re
from itertools import groupby
import emoji
from pythainlp.util import countthai
from tqdm import tqdm

def rm_reptitive(text):
    s = ""
    groups = groupby(text)
    for label, group in groups:
        g = list(group)
        if len(g) >= 3:
            s += f"{label*3} rep "
        else:
            s += "".join(g)
    return s

def remove_space(sent):
    newwords = []
    for w in sent:
        if len(w.strip())==0:
            continue
        newwords.append(w)
    return newwords


import re
def notthai(w):
    if countthai(w) > 50:
        return False
    
    if w in ["usr", "sys", "rep"]:
        return False
    
    nt = re.sub(r'\W+', '', w)
    if len(nt) > 0 and not nt.isnumeric():
#         print(nt)
        return True
    return False

def analyse_conv_per_person(texts):
    
    # Word Statistic
    texts = [t.lower() for t in texts]
    texts = [rm_reptitive(t) for t in texts]
    words = [word_tokenize(t) for t in texts]
    words = [remove_space(w) for w in words]
    
    nlongword = 0
    ndict = 0
    nthai = 0
    nnotthai = 0
    for sent in words:
        ndict += sum([1 if w in thaidict_royal else 0 for w in sent])
        nlongword += sum([1 if len(w) > 7 else 0 for w in sent])
        nthai += sum([1 if countthai(w) > 50 else 0 for w in sent])
            
        nnotthai += sum([1 if notthai(w) else 0 for w in sent])
        
        
    
    uwords = set()
    for sent in words:
        uwords.update(sent)
    
    # Lexicon 
    lex = []
    for sidx, sent in enumerate(words):
        for widx, w in enumerate(sent):
            if w not in lexicons_keys:
                continue 
            
            s = "".join(sent[widx:])
            for l in lexicons_keys[w]:
                if not s.startswith(l):
                    continue

                lex.extend(lexicons[l])
#                 print(">>", w, l, lexicons[l])

    lexcat = {}
    for l in lex:
        if l not in lexcat:
            lexcat[l] = 0
        lexcat[l] += 1
    
    # Stylistic words
    nrepeat = 0
    for sidx, sent in enumerate(words):
        nrepeat += sum([1 if w=="rep" else 0 for w in sent])
    
    s = " ".join(texts)
    nemoji = emoji.emoji_count(s)
    

    return {
        "nsent": len(texts),
        "nword": sum([len(w) for w in words]),
        "ndict": ndict,
        "nunique": len(uwords),
        "nlongword": nlongword,
        "nrepeat": nrepeat,
        "nthai": nthai,
        "nnotthai": nnotthai,
        "nemoji": nemoji,
        **lexcat
    }
  


def analyse_conversation(df):
    metrics = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        ms = analyse_conv_per_person(row["sys"])
        mu = analyse_conv_per_person(row["user"])
        metrics.append((ms, mu))
    
    return metrics

analyse_conv_per_person(["เมิงงงงงงงมันโง่เหมือนควายยยยยยยยย"]);

## Run Analysis

In [75]:
import pickle
def save_obj_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)
    
def load_obj_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

In [76]:
analysis_values = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

analysis_labels = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

In [78]:
# df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][0] = metrics
# analysis_labels["closeness"][0] = df["label"].values


# df = get_task2_conver("./Task2/annotated/annotated.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][1] = metrics
# analysis_labels["closeness"][1] = df["label"].values


# df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["closeness"][2] = metrics
# analysis_labels["closeness"][2] = df["label"].values

In [79]:
# df = get_task1_conver("./Task1/annotated_conersations.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][0] = metrics
# analysis_labels["authority"][0] = df["label"].values


# df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][1] = metrics
# analysis_labels["authority"][1] = df["label"].values


# df = get_task1_conver("./Task3/annotated/annotated.jsonl", "authority", skips = [])
# metrics = analyse_conversation(df)
# analysis_values["authority"][2] = metrics
# analysis_labels["authority"][2] = df["label"].values

In [80]:
# save_obj_values("analysis_values.pkl", analysis_values)
# save_obj_values("analysis_labels.pkl", analysis_labels)

In [81]:
analysis_values = load_obj_values("analysis_values.pkl")
analysis_labels = load_obj_values("analysis_labels.pkl")

In [82]:
metric_names = {
    "Conversational Statistics" : {
        "nsent": "Number of utterance",
        "nword": "Number of word",
    },
    "Linguistic Complexity" : {
        "nunique": "Vocabulary size",
        "nthai": "Thai words",
        "nnotthai": "Non-Thai words",
        "nlongword": "Long words",
        "ndict": "Dictionary words",
        "transliterated": "Transliteration",
    },
    "Pronoun": {
        "pronoun": "All pronoun",
        "pronoun_1st": ">> 1st person pronoun",
        "pronoun_2nd": ">> 2nd person pronoun",
        "pronoun_3rd": ">> 3rd person pronoun",
        "pronoun_misspelling": ">> Pronoun in non-standard spelling",
    },
    
    "Sentence-ending Particles": {
        "particles": "All particles",
        "particles_SARP": ">> Socially-related particles",
        "particles_notSARP": ">> Non-socially-related particles",
        "particles_misspelling": ">> Particle in non-standard spelling",
    },
    
    "Sentiment-related": {
        "sentiment": "Sentiment words",
        "sentiment_positive": ">> Positive words",
        "sentiment_negative": ">> Negative words",
    },
    
    "Spelling Variation": {
        "misspelling": "All spelling variation",
        "misspelling_common": ">> Common misspelt words",
        "misspelling_intention": ">> Morphophonemic variation",
        "misspelling_shorten": ">> Simplified variation",
        "nrepeat": ">> Repeated characters",
        "nemoji": ">> Emoji",
#         "abbr": "Abbreviation",
#         "slang": "Slang",
#         "swear": "Swear words"
    }
}

In [83]:
clse_print_labels = ['1. Close', '2. Know each other', "3. Don't know each other", "4. Don't like each other"]
auth_print_labels = ['0. Very respect', '1. Respect', '2. Normal', '3. Not respect']

## Linear Regression

In [85]:
def to_features(analysis_values, analysis_labels, factor, setting, to_vec_func, skips=[]):
    values = analysis_values[factor][setting]
    labels = analysis_labels[factor][setting]
    assert(len(values)==len(labels))
    
    rows = []
    for (ms, mu), l in zip(values, labels):
        if l in skips:
            continue
        
        x = {}
        for m in mu:
            if m in ["nsent", "nword"]:
                continue
            x[m] = mu[m]*100/mu["nword"]
        
        if "particles_SARP" in x:
            x["particles_notSARP"] = x["particles"] - x["particles_SARP"]
                
        y = to_vec_func(l)
        rows.append({
            **x,
            **y
        })
            
    feats = pd.DataFrame(rows)
    feats.dropna(subset=["y"], inplace=True)
    feats.drop(columns=['particles_??', "slang", "abbr", "swear"], inplace=True)  ## drop unused features
    feats.drop(columns=['sentiment', "misspelling", "particles", "pronoun"], inplace=True) ## drop overal features
    feats.fillna(0, inplace=True)
    return feats

In [86]:
import statsmodels.api as sm
import numpy as np

def print_anova_test(feats):
    x_columns = [col for col in feats.columns if col!="y"]
    X = feats[x_columns]
    X = sm.add_constant(X)
    Y = feats["y"]
    model = sm.OLS(Y,X)
    results = model.fit()

    
    ncoef = len(x_columns)+1
    
    t_test = results.t_test(np.identity(ncoef))
    f_test = results.f_test(np.identity(ncoef))
    
    weights = []
    for i, col in enumerate(["bias"]+x_columns):
        o = {
            "feat": col, 
            "coef": t_test.effect[i], 
            "pval": t_test.pvalue[i]
        }

        weights.append(o)
    
    outputs = pd.DataFrame(weights)
    return outputs

In [87]:
# def print_weights(outputs, coef_labels, labels, base_category=None):
    
#     coefs = {}
#     non_coefs = {}
#     for i, label in enumerate(coef_labels):
#         if i==0:
#             continue
            
#         cond = outputs[f"pval{i}"] < 0.05
#         for _, row in outputs[cond].sort_values(f"coef{i}", ascending=False).iterrows():
#             coefs[(label, row["feat"])] = row[f"coef{i}"]
        
#         cond = outputs[f"pval{i}"] >= 0.05
#         for _, row in outputs[cond].iterrows():
#             non_coefs[(label, row["feat"])] = row[f"coef{i}"]

# #     print(coefs)
#     for g in metric_names:
#         print("\multicolumn{5}{l}{\\textit{"+g+"}} \\\\")
#         for m in metric_names[g]:
#             s = f"{metric_names[g][m]} "
#             for l in labels:
#                 if l == base_category:
#                     s += f"& * "
#                 elif (l, m) in coefs:
#                     val = coefs[(l, m)]
#                     s += "& \cellcolor{gray!25} "+f"{val:.2f} "
#                 elif (l, m) in non_coefs:
#                     val = non_coefs[(l, m)]
#                     s += f"& {val:.2f} "
#                 else:
#                     s += f"& - "

#             s += "\\\\"
#             print(s)
#         print("&  & &  & \\\\")
#         print("\hline")

# # gray!25 < gray!50 < gray!80

In [88]:
def closeness_to_vec1(label):
    if label == "1. Close":
        return {"y": 1}
    elif label == "2. Know each other":
        return {"y": 0.5}
    elif label == "3. Don't know each other": 
        return {"y": 0}
    elif label == "4. Don't like each other":
        return {}
    else:
        return {}

feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec1, skips=[])
clos1_weights = print_anova_test(feats)
clos1_weights[clos1_weights["pval"]<0.05]

,feat,coef,pval
0,bias,-1.189055,2.154550e-03
1,ndict,-0.009350,2.667411e-05
2,nunique,0.006037,1.961348e-06
3,nlongword,-0.015214,7.956090e-04
5,nthai,0.023700,5.941378e-10
8,sentiment_positive,-0.006051,1.369086e-02
12,pronoun_3rd,0.025668,3.394937e-05
15,particles_SARP,-0.016895,2.041104e-24
16,particles_notSARP,-0.020475,1.252328e-08


In [89]:
feats.columns

Index(['ndict', 'nunique', 'nlongword', 'nrepeat', 'nthai', 'nnotthai',
       'nemoji', 'sentiment_positive', 'misspelling_shorten',
       'misspelling_intention', 'pronoun_2nd', 'pronoun_3rd', 'pronoun_1st',
       'sentiment_negative', 'y', 'particles_SARP', 'particles_notSARP',
       'transliterated', 'particles_misspelling', 'misspelling_common',
       'pronoun_misspelling'],
      dtype='object')

In [90]:
def authority_to_vec1(label):
    if label == '0. Very respect':
        return {"y":1}
    elif label == '1. Respect': 
        return {"y":0.5}
    elif label == '2. Normal':
        return {"y":0}
    elif label == '3. Not respect':
        return {}
    else:
        return {}

feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec1, skips=["3. Not respect"])
auth1_weights = print_anova_test(feats)
auth1_weights[auth1_weights["pval"]<0.05]

,feat,coef,pval
2,nunique,-0.003508,0.004878
4,nrepeat,0.028485,0.000115
11,pronoun_2nd,-0.009892,0.044444
14,sentiment_negative,-0.011918,0.025318
15,particles_SARP,0.004521,0.004808
17,transliterated,-0.031692,0.047050


In [91]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec1, skips=[])
clos2_weights = print_anova_test(feats)
clos2_weights[clos2_weights["pval"]<0.05]

,feat,coef,pval
0,bias,-0.383416,1.666208e-04
2,nunique,0.002139,1.278620e-03
4,nrepeat,0.014329,6.433758e-10
5,nthai,0.004042,4.558186e-04
6,nnotthai,0.006106,4.844615e-05
7,nemoji,0.003191,7.771816e-04
9,misspelling_intention,0.001958,2.173902e-02
10,pronoun_1st,0.006063,2.305669e-02
17,sentiment_positive,0.003102,8.144852e-03


In [92]:
def authority_to_vec2(label):
    if label == '0. Very respect':
        return {}
    elif label == '1. Respect': 
        return {"y":1}
    elif label == '2. Normal':
        return {"y":0.5}
    elif label == '3. Not respect':
        return {"y":0}
    else:
        return {}

feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec2)
auth2_weights = print_anova_test(feats)
auth2_weights[auth2_weights["pval"]<0.05]

,feat,coef,pval
0,bias,0.709841,3.610281e-24
1,ndict,-0.002766,3.980655e-09
4,nrepeat,-0.003256,2.670828e-02
7,nemoji,0.004648,3.240347e-12
8,misspelling_shorten,-0.002203,3.287505e-04
9,misspelling_intention,0.002784,9.779449e-07
10,particles_SARP,0.006343,2.775427e-09
11,sentiment_negative,-0.006525,9.490411e-13
12,sentiment_positive,0.006605,2.852760e-16
13,particles_notSARP,0.004785,4.597548e-03


In [93]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec1, skips=[])
clos3_weights = print_anova_test(feats)
clos3_weights[clos3_weights["pval"]<0.05]

,feat,coef,pval
1,ndict,-0.005251,1.109585e-04
2,nunique,0.002181,4.712851e-03
3,nlongword,-0.012976,2.904897e-06
4,nrepeat,0.011652,7.029028e-03
5,nthai,0.013428,8.385601e-09
8,sentiment_positive,-0.004750,1.522921e-03
12,pronoun_3rd,0.013399,4.520069e-04
13,pronoun_1st,0.012760,8.521153e-04
14,sentiment_negative,0.010084,2.410188e-03
15,particles_SARP,-0.009866,1.351083e-22


In [94]:
feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec2)
auth3_weights = print_anova_test(feats)
auth3_weights[auth3_weights["pval"]<0.05]

,feat,coef,pval
0,bias,1.217911,1.170197e-13
2,nunique,0.001186,2.510124e-02
3,nlongword,0.006385,7.835378e-04
4,nrepeat,-0.018801,3.272427e-10
5,nthai,-0.008090,4.364052e-07
6,nnotthai,-0.015874,4.191689e-05
8,sentiment_positive,0.003289,1.401141e-03
9,misspelling_shorten,-0.004453,1.960161e-05
13,pronoun_1st,-0.005656,2.692381e-02
15,particles_SARP,0.005261,2.020995e-14


In [95]:
# clos3_weights

### Print Results

In [96]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

In [97]:
def outputs_to_dict(outputs):
    coefs = {}
    
    for _, row in outputs.iterrows():
        coefs[row["feat"]] = (row["coef"], row["pval"])

    return coefs

In [98]:
printed_text = "" 
printed_text += "\subsection{Closeness}"+"\n"
outputs = [
    outputs_to_dict(clos1_weights),
    outputs_to_dict(clos2_weights),
    outputs_to_dict(clos3_weights)
]

printed_text += "\\begin{longtable}[h]{"+"\n"
printed_text += "    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|"+"\n"

printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"

printed_text += "}"+"\n"
printed_text += "    \hline"+"\n"
printed_text += "    Lexical Features & Setting 1  & Setting 2 & Setting 3 \\\\"+"\n"
printed_text += "    \hline"+"\n"
#     printed_text += "    \endfirsthead"+"\n"
#     printed_text += ""+"\n"
printed_text += "    \endhead"+"\n"
printed_text += ""+"\n"

# for sec, results in zip(sections, outputs):
for g in metric_names:
    if g in ["Conversational Statistics"]:
        continue

    printed_text += "    \multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
    printed_text += "    \hline"+"\n"

    for m in metric_names[g]:
        s = f"        {metric_names[g][m]} "
        for out in outputs:
            if m not in out:
                s += f"& - "
            else:
                val, pval = out[m]

                if pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2e} "
                else:
                    s += f"& {val:.2e} "
        s += "\\\\"
        # print(s)
        printed_text += s+"\n"


    printed_text += "        &  & &\\\\"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += ""+"\n"
printed_text += "\label{closeness_linear_weights}"+"\n"
printed_text += "\end{longtable}"+"\n"
printed_text += "\clearpage"+"\n"



In [99]:
printed_text += "\subsection{Respect}"+"\n"
outputs = [
    outputs_to_dict(auth1_weights),
    outputs_to_dict(auth2_weights),
    outputs_to_dict(auth3_weights)
]

printed_text += "\\begin{longtable}[h]{"+"\n"
printed_text += "    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|"+"\n"

printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"

printed_text += "}"+"\n"
printed_text += "    \hline"+"\n"
printed_text += "    Lexical Features & Setting 1  & Setting 2 & Setting 3 \\\\"+"\n"
printed_text += "    \hline"+"\n"
#     printed_text += "    \endfirsthead"+"\n"
#     printed_text += ""+"\n"
printed_text += "    \endhead"+"\n"
printed_text += ""+"\n"

# for sec, results in zip(sections, outputs):
for g in metric_names:
    if g in ["Conversational Statistics"]:
        continue

    printed_text += "    \multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
    printed_text += "    \hline"+"\n"

    for m in metric_names[g]:
        s = f"        {metric_names[g][m]} "
        for out in outputs:
            if m not in out:
                s += f"& - "
            else:
                val, pval = out[m]

                if pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2e} "
                else:
                    s += f"& {val:.2e} "
        s += "\\\\"
        # print(s)
        printed_text += s+"\n"


    printed_text += "        &  & &\\\\"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += ""+"\n"
printed_text += "\label{closeness_linear_weights}"+"\n"
printed_text += "\end{longtable}"+"\n"



In [101]:
print(printed_text)

\subsection{Closeness}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
}
    \hline
    Lexical Features & Setting 1  & Setting 2 & Setting 3 \\
    \hline
    \endhead

    \multicolumn{4}{l}{\textit{Linguistic Complexity}} \\
    \hline
        Vocabulary size & \cellcolor{gray!25} 6.04e-03 & \cellcolor{gray!25} 2.14e-03 & \cellcolor{gray!25} 2.18e-03 \\
        Thai words & \cellcolor{gray!25} 2.37e-02 & \cellcolor{gray!25} 4.04e-03 & \cellcolor{gray!25} 1.34e-02 \\
        Non-Thai words & 1.17e-02 & \cellcolor{gray!25} 6.11e-03 & 8.24e-04 \\
        Long words & \cellcolor{gray!25} -1.52e-02 & -1.74e-03 & \cellcolor{gray!25} -1.30e-02 \\
        Dictionary words & \cellcolor{gray!25} -9.35e-03 & -1.19e-03 & \cellcolor{gray!25} -5.25e-03 \\
        Transliteration & -6.47e-03 & -3.32e-03 & -7.13e-03 \\
        &  & &\\
    \hline

    \m

### Evaluate Linear Model

In [83]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error, r2_score

from random import random

def get_data_from_df_to_data(df):
    x_columns = [col for col in df.columns if col!="y"]
    X = df[x_columns]
    X = sm.add_constant(X)
    Y = df["y"]
    
    return X, Y
    
def train_model(feats, label_fn, random_state=42, random_guess=False):
    d = feats.sample(frac=1, random_state=random_state)
    train, test = np.split(d, [int(0.9*len(d))])
#     train, validate, test = np.split(d, [int(0.8*len(d)), int(0.9*len(d))])
    
    Xtrain, Ytrain = get_data_from_df_to_data(train)
    model = sm.OLS(Ytrain,Xtrain)
    M = model.fit()
    
#     Xval, Yval = get_data_from_df_to_data(validate)
#     thesholds = tune_thesholds(M, Xval, Yval)
        
    Xtest, Ytest = get_data_from_df_to_data(test)

    predictions = M.predict(Xtest)
    
    predicted_labels = [label_fn(p) for p in predictions]
    actual_labels = [label_fn(p) for p in Ytest]
    p, r, f1, _ = precision_recall_fscore_support(actual_labels, predicted_labels, average='macro', zero_division=0)
    
#     print((np.array(predicted_labels)==np.array(actual_labels)).sum())
    return {
        "rmse": np.sqrt(mean_squared_error(Ytest, predictions)),
        "r2": r2_score(Ytest, predictions),
        
        "accuracy": accuracy_score(actual_labels, predicted_labels),
        "f1": f1,
        "precision": p,
        "recall": r,
    }

In [72]:
def print_accuracy(feats, label_fn, random_guess=False):
    eval_metrics = {}
    for i in range(100):
        metrics = train_model(feats, label_fn, random_state=i, random_guess=random_guess)
        for k in metrics:
            if k not in eval_metrics:
                eval_metrics[k] = []
            eval_metrics[k].append(metrics[k])

    for k in eval_metrics:
        print(f"{k}: {np.mean(eval_metrics[k]):.3f} ± {np.std(eval_metrics[k]):.3f}")

In [73]:
random_guess=False

In [74]:
def closeness_label_fn(label):
    if label == '1. Close':
        return 1
    elif label =='2. Know each other':
        return 0.5
    elif label == "3. Don't know each other":
        return 0
    elif type(label)==str:
        assert(False)
    
    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>
    
    if label > 0.66:
        return '1. Close'
    elif label > 0.33:
        return '2. Know each other'
    else:
        return "3. Don't know each other"
    
feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec1, skips=[])
print_accuracy(feats, closeness_label_fn, random_guess=random_guess)

rmse: 0.405 ± 0.018
r2: 0.173 ± 0.064
accuracy: 0.408 ± 0.042
f1: 0.400 ± 0.040
precision: 0.533 ± 0.046
recall: 0.435 ± 0.042


In [84]:
def authority_label_fn(label):
    if label == '0. Very respect':
        return 1
    elif label =='1. Respect':
        return 0.5
    elif label == "2. Normal":
        return 0
    elif type(label)==str:
        assert(False)
    
    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>
    
    if label > 0.66:
        return '0. Very respect'
    elif label > 0.33:
        return '1. Respect'
    else:
        return "2. Normal"
    
feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec1, skips=[])
print_accuracy(feats, authority_label_fn, random_guess=random_guess)

rmse: 0.395 ± 0.017
r2: 0.013 ± 0.051
accuracy: 0.450 ± 0.038
f1: 0.314 ± 0.028
precision: 0.313 ± 0.083
recall: 0.364 ± 0.033


In [85]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec1, skips=[])
print_accuracy(feats, closeness_label_fn, random_guess=random_guess)

rmse: 0.331 ± 0.023
r2: 0.035 ± 0.050
accuracy: 0.773 ± 0.029
f1: 0.327 ± 0.027
precision: 0.419 ± 0.151
recall: 0.346 ± 0.019


In [86]:
def authority2_label_fn(label):
    if label == '1. Respect':
        return 1
    elif label =='2. Normal':
        return 0.5
    elif label == "3. Not respect":
        return 0
    elif type(label)==str:
        assert(False)
    
    if label > 0.66:
        return '1. Respect'
    elif label > 0.33:
        return '2. Normal'
    else:
        return "3. Not respect"
    
feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec2)
print_accuracy(feats, authority2_label_fn, random_guess=random_guess)

rmse: 0.248 ± 0.011
r2: 0.153 ± 0.048
accuracy: 0.704 ± 0.029
f1: 0.444 ± 0.038
precision: 0.572 ± 0.069
recall: 0.430 ± 0.028


In [87]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec1, skips=[])
print_accuracy(feats, closeness_label_fn, random_guess=random_guess)

rmse: 0.247 ± 0.016
r2: 0.203 ± 0.080
accuracy: 0.629 ± 0.048
f1: 0.542 ± 0.087
precision: 0.602 ± 0.121
recall: 0.544 ± 0.079


In [88]:
feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec2)
print_accuracy(feats, authority2_label_fn, random_guess=random_guess)

rmse: 0.165 ± 0.019
r2: 0.119 ± 0.131
accuracy: 0.883 ± 0.029
f1: 0.463 ± 0.065
precision: 0.543 ± 0.122
recall: 0.447 ± 0.058
